In [13]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
from PIL import Image
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor ,SegformerFeatureExtractor
import pandas as pd
import cv2
import numpy as np
import albumentations as aug
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchinfo import summary
import numpy as np
from datasets import load_metric
import evaluate

In [14]:
class ImageSegmentationDataset(Dataset):
    """Image segmentation dataset."""
    def __init__(self, root_dir, feature_extractor, transforms=None, train=True):
        super(ImageSegmentationDataset,self).__init__()
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.train = train
        self.transforms = transforms
        self.img_dir = os.path.join(self.root_dir, "images")
        self.ann_dir = os.path.join(self.root_dir, "pngmasks")
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
            image_file_names.extend(files)
        self.images = sorted(image_file_names)
        
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
            annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), "There must be as many images as there are segmentation maps"


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = cv2.imread(os.path.join(self.img_dir, self.images[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        segmentation_map = cv2.imread(os.path.join(self.ann_dir, self.annotations[idx]))
        segmentation_map = cv2.cvtColor(segmentation_map, cv2.COLOR_BGR2GRAY)
        if self.transforms is not None:
            augmented = self.transforms(image=image, mask=segmentation_map)
            encoded_inputs = self.feature_extractor(augmented['image'], augmented['mask'], return_tensors="pt")
        else:
            encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_()

        return encoded_inputs

In [15]:
transform = aug.Compose([
    aug.Flip(p=0.5)
],is_check_shapes=False)

In [16]:
train_dir =r"D:\graval detection project\datasets\under_water_masks_dataset\train"
valid_dir=r"D:\graval detection project\datasets\under_water_masks_dataset\val"
test_dir=r"D:\graval detection project\datasets\under_water_masks_dataset\test"
feature_extractor = SegformerImageProcessor.from_pretrained ("nvidia/mit-b0")
train_dataset = ImageSegmentationDataset(root_dir=train_dir, feature_extractor=feature_extractor, transforms=transform)
valid_dataset = ImageSegmentationDataset(root_dir=valid_dir, feature_extractor=feature_extractor, transforms=None, train=False)
test_dataset = ImageSegmentationDataset(root_dir=test_dir, feature_extractor=feature_extractor, transforms=None, train=False)

In [17]:
classes = ["stone"]
print(classes)
id2label = {0:classes[0]}
print(id2label)
label2id = {v: k for k, v in id2label.items()}
print(label2id)

['stone']
{0: 'stone'}
{'stone': 0}


In [18]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True,num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=4,shuffle=False,num_workers=0)
test_dataloader  = DataLoader(test_dataset,batch_size=4,shuffle=False,num_workers=0)

In [19]:
dataiter = iter(train_dataloader)


In [20]:
class SegformerFinetuner(pl.LightningModule):
    
    def __init__(self, id2label, train_dataloader=None, val_dataloader=None, test_dataloader=None, metrics_interval=100):
        super(SegformerFinetuner, self).__init__()
        self.id2label = id2label
        self.metrics_interval = metrics_interval
        self.train_dl = train_dataloader
        self.val_dl = val_dataloader
        self.test_dl = test_dataloader
        self.num_classes = len(id2label.keys())
        self.label2id = {v:k for k,v in self.id2label.items()}
        self.model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0", ignore_mismatched_sizes=True,
                                                         reshape_last_stage=True)
        self.train_mean_iou = evaluate.load("mean_iou")
        self.val_mean_iou = evaluate.load("mean_iou")
        self.test_mean_iou = evaluate.load("mean_iou")
        
    def forward(self, images, masks):
        outputs = self.model(pixel_values=images, labels=masks)
        return(outputs)
    
    def training_step(self, batch, batch_nb):
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(logits, size=masks.shape[-2:], mode="bilinear", align_corners=False)
        predicted = upsampled_logits.argmax(dim=1)
        self.train_mean_iou.add_batch(predictions=predicted.detach().cpu().numpy(), references=masks.detach().cpu().numpy())
        if batch_nb % self.metrics_interval == 0:
            metrics = self.train_mean_iou.compute(num_labels=self.num_classes, ignore_index=255, reduce_labels=False,)
            metrics = {'loss': loss, "mean_iou": metrics["mean_iou"], "mean_accuracy": metrics["mean_accuracy"]}
            for k,v in metrics.items():
                self.log(k,v)
            return(metrics)
        else:
            return({'loss': loss})
    def validation_step(self, batch, batch_nb):
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(logits, size=masks.shape[-2:], mode="bilinear", align_corners=False)
        predicted = upsampled_logits.argmax(dim=1)
        self.val_mean_iou.add_batch(predictions=predicted.detach().cpu().numpy(), references=masks.detach().cpu().numpy())
        return({'val_loss': loss})
    
    
    def test_step(self, batch, batch_nb):
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(logits, size=masks.shape[-2:], mode="bilinear", align_corners=False)
        predicted = upsampled_logits.argmax(dim=1)
        self.test_mean_iou.add_batch(predictions=predicted.detach().cpu().numpy(), references=masks.detach().cpu().numpy())
        return({'test_loss': loss})
    
    
    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=2e-05, eps=1e-08)
    
    def train_dataloader(self):
        return self.train_dl
    
    def val_dataloader(self):
        return self.val_dl
    
    def test_dataloader(self):
        return self.test_dl

In [21]:
SegformerFineTuner=SegformerFinetuner(id2label=id2label,train_dataloader=train_dataloader,val_dataloader=valid_dataloader,test_dataloader=test_dataloader,metrics_interval=10)
SegformerFineTuner.to(device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.weight', 'decode_head.classifier.weight', 'decode_head.classifier.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerFinetuner(
  (model): SegformerForSemanticSegmentation(
    (segformer): SegformerModel(
      (encoder): SegformerEncoder(
        (patch_embeddings): ModuleList(
          (0): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
            (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          )
          (1): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
          (2): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
          )
          (3): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (la

In [22]:
from torchinfo import summary
summary(model=SegformerFineTuner)

Layer (type:depth-idx)                                                           Param #
SegformerFinetuner                                                               --
├─SegformerForSemanticSegmentation: 1-1                                          --
│    └─SegformerModel: 2-1                                                       --
│    │    └─SegformerEncoder: 3-1                                                3,319,392
│    └─SegformerDecodeHead: 2-2                                                  --
│    │    └─ModuleList: 3-2                                                      132,096
│    │    └─Conv2d: 3-3                                                          262,144
│    │    └─BatchNorm2d: 3-4                                                     512
│    │    └─ReLU: 3-5                                                            --
│    │    └─Dropout: 3-6                                                         --
│    │    └─Conv2d: 3-7                              

In [23]:
trainer=pl.Trainer(max_epochs=10,val_check_interval=len(valid_dataloader),accelerator="gpu",devices=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [24]:
trainer.fit(model=SegformerFineTuner,train_dataloaders=train_dataloader,val_dataloaders=valid_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                             | Params
-----------------------------------------------------------
0 | model | SegformerForSemanticSegmentation | 4.0 M 
-----------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.885    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/